# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

C:\Users\htrujillo\Udacity\projects\data-engineering\02-Data-Modeling-with-Cassandra\Sparkify-Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

#### Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
def create_keyspace(
        keyspace: str,
        session: cassandra.cluster.Session
) -> int:
    """This function creates a cassandra keyspace.

    Args:
        keyspace: keyspace name.
        session: Cassandra's cluster session.

    """
    query = f"CREATE KEYSPACE IF NOT EXISTS {keyspace}"
    query += " WITH REPLICATION = "
    query += "{'class': 'SimpleStrategy', 'replication_factor': 1}"
    try:
        session.execute(query)
    except Exception as e:
        print(e)

    return 0

create_keyspace(
    keyspace='sparkify',
    session=session
)

0

#### Set Keyspace

In [7]:
def set_keyspace(
        keyspace: str,
        session: cassandra.cluster.Session
) -> int:
    """This function set the session' keyspace.

    Args:
        keyspace: keyspace name.
        session: Cassandra's cluster session.

    """
    try:
        session.set_keyspace(keyspace)
    except Exception as e:
        print(e)

    return 0

set_keyspace(
    keyspace='sparkify',
    session=session
)

0

### Data Modeling with Cassandra.

#### Create queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Python Classes

To facilitate the Data modeling and database querying, I have created two classes:

* CassandraTable: This class provides an interface based on Python's Builder Pattern to facilitate table creation.
* Query: This class provides an interface based on Python's Builder Pattern to facilitate query creation and execution.

In [8]:
class CassandraTable:
    """This Class represents a Cassandra's Table.

        Args:
            session: Cassandra's cluster session.
            table_name : Table Name.

        Attributes:
            session: Cassandra's cluster session.
            table_name : Table Name.
            columns: Table columns.
            columns_types: Table columns with types.
            primary_key: Primary kye definition.

        """

    def __init__(
            self,
            session: cassandra.cluster.Session,
            table_name: str) -> None:

        self.session = session
        self.columns: str = ''
        self.columns_types: str = ''
        self.table_name: str = table_name
        self.primary_key: str = ''

    def define_columns(self, columns: dict[str: str]):
        """This method generates the column definition.

            Args:
                columns: column definition dict where:
                    keys: column name.
                    values: column type.
        """
        cols = [col for col in columns.keys()]
        columns_types = [col + ' ' + col_type for col, col_type in columns.items()]

        cols = ', '.join(cols)
        columns_types = ', '.join(columns_types)

        self.columns = cols
        self.columns_types = columns_types

        return self

    def define_primary_key(self, primary_key: dict[str, list[str]]):
        """This method generates the primary key definition.

            Args:
                primary_key: column definition dict where:
                    keys: type of key: partition or clustering.
                    values: columns.
        """

        partition = primary_key.get('partition', None)
        partition = ', '.join(partition)

        clustering = primary_key.get('clustering', '')
        if clustering != '':
            clustering = ', '.join(clustering)

            self.primary_key = partition + ', ' + clustering

        else:

            self.primary_key = partition

        return self

    def build(self) -> str:
        """This method generates the create query."""

        query = f"CREATE TABLE IF NOT EXISTS {self.table_name}"
        query += '(' + self.columns_types + ', '
        query += f'PRIMARY KEY ({self.primary_key})' + ')'

        return query

    def execute(self, echo: bool = True) -> None:
        """This method executes the create query."""

        query = self.build()

        if echo:
            print(query)

        try:
            rows = self.session.execute(query)
        except Exception as e:
            print(e)

        return rows

    def drop(self) ->  None:
        """This method drops the table."""

        query = f'DROP TABLE {self.table_name}'

        try:
            session.execute(query)
        except Exception as e:
            print(e)

            
class Query:
    """This Class represents a Cassandra's Query statement.

        Args:
            session: Cassandra's cluster session.

        Attributes:
            session: Cassandra's cluster session.
            columns: Selected columns.

        """

    def __init__(
            self,
            session: cassandra.cluster.Session
    ) -> None:

        self.session = session
        self.columns: str = ''
        self.table_name: str = ''
        self.where_value: str = ''

    def select(self, columns: str):
        """This method is used to select columns.

            Args:
                columns: column to be selected
        """

        self.select_columns = columns

        return self

    def from_table(self, table_name: str):
        """This method is used to define table name.

            Args:
                table_name: Table name.
        """
        self.table_name = table_name

        return self

    def where(self, conditions: str):
        """This method is used to define WHERE Clause.

            Args:
                conditions: hardcoded where conditions.
        """
        self.where_value = conditions

        return self

    def build(self) -> str:
        """This method generates the select query."""

        where_clause = ''

        if self.where_value:
            where_clause = f"WHERE {self.where_value}"

        return f"SELECT {self.select_columns} FROM {self.table_name} {where_clause}"

    def execute(self):
        """This method executes the create query."""

        query = self.build()

        try:
            rows = self.session.execute(query)
        except Exception as e:
            print(e)

        return rows


#### Query 1: Songs info by Session

> Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

To fulfill this business requirement, we are going to create the following table structure:

* Columns:
    * `artirs`
    * `iteminsession`
    * `length`
    * `sessionid`
    * `song`
* Primary Key:
    * Partition Key: `sessionid`
    * Clustering column: `iteminsession`

Since we are going to filter by sessionid and iteminsession, we'll need to have both as a composed key. 

In [9]:
songs_by_session_table = CassandraTable(
    session=session,
    table_name='songs_by_session'
)

songs_by_session_table.\
    define_columns(
        columns={
            'artist': 'text',
            'iteminsession': 'int',
            'length': 'float',
            'sessionid': 'int',
            'song': 'text',
        }).\
    define_primary_key(
        primary_key={
            'partition': ['sessionid'],
            'clustering': ['iteminsession']
        }).\
    execute()

CREATE TABLE IF NOT EXISTS songs_by_session(artist text, iteminsession int, length float, sessionid int, song text, PRIMARY KEY (sessionid, iteminsession))


In [10]:
file = 'event_datafile_new.csv'


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = f"INSERT INTO {songs_by_session_table.table_name} "
        query += '(' + songs_by_session_table.columns + ')'
        num_cols = len(songs_by_session_table.columns.split(','))
        query += "VALUES (" + ('%s, ' * num_cols)[:-2] + ')'
        session.execute(query, (line[0], int(line[3]), float(line[5]), int(line[8]), line[9]))

#### SELECT to verify that the data have been inserted

In [12]:
query = Query(
    session=session
)

rows = query.\
    select(
        columns="artist, song, length").\
    from_table(
    table_name=songs_by_session_table.table_name).\
    where(
    conditions="sessionid=338 and iteminsession=4").\
    execute()

for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Query 2: Songs played by user

> Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

To fulfill this business requirement, we are going to create the following table structure:

* Columns:
    * `artirs`
    * `song`
    * `userid`
    * `iteminsession`
    * `first_name`
    * `last_name`
    * `sessionid`
* Primary Key:
    * Partition Key: `userid`
    * Clustering column: `sessionid`, `iteminsession`
    
In this case we have included iteminsession as clustering column to sort the table by this column. 

In [13]:
artist_song_user = CassandraTable(
    session=session,
    table_name='artist_song_user'
)

artist_song_user.\
    define_columns(
        columns={
            'artist': 'text',
            'song': 'text',
            'userid': 'int',
            'iteminsession': 'int',
            'first_name': 'text',
            'last_name': 'text',
            'sessionid': 'int'
        }).\
    define_primary_key(
        primary_key={
            'partition': ['userid'],
            'clustering': ['sessionid', 'iteminsession']
        }).\
    execute()

CREATE TABLE IF NOT EXISTS artist_song_user(artist text, song text, userid int, iteminsession int, first_name text, last_name text, sessionid int, PRIMARY KEY (userid, sessionid, iteminsession))


In [14]:
file = 'event_datafile_new.csv'


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = f"INSERT INTO {artist_song_user.table_name} "
        query += '(' + artist_song_user.columns + ')'
        num_cols = len(artist_song_user.columns.split(','))
        query += "VALUES (" + ('%s, ' * num_cols)[:-2] + ')'
        session.execute(query, (line[0], line[9], int(line[10]), int(line[3]), line[1], line[4], int(line[8])))

#### SELECT to verify that the data have been inserted

In [15]:
query = Query(
    session=session
)

rows = query.\
    select(
        columns="artist, song, first_name, last_name").\
    from_table(
    table_name=artist_song_user.table_name).\
    where(
    conditions="userid = 10 and sessionid = 182").\
    execute()

for row in rows:
    print(row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Query 3: Songs played by user

> Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

To fulfill this business requirement, we are going to create the following table structure:

* Columns:
    * `userid`
    * `first_name`
    * `last_name`
    * `song`
* Primary Key:
    * Partition Key: `song`
    * Clustering column: `userid`
    
In this case, we have included userid as a clustering column to avoid overwriting the song by a user.

In [16]:
user_song_table = CassandraTable(
    session=session,
    table_name='user_song_table'
)

user_song_table.\
    define_columns(
        columns={
            'userid': 'int',
            'first_name': 'text',
            'last_name': 'text',
            'song': 'text'
        }).\
    define_primary_key(
        primary_key={
            'partition': ['song'],
            'clustering': ['userid']
        }).\
    execute()                    

CREATE TABLE IF NOT EXISTS user_song_table(userid int, first_name text, last_name text, song text, PRIMARY KEY (song, userid))


In [17]:
file = 'event_datafile_new.csv'


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = f"INSERT INTO {user_song_table.table_name} "
        query += '(' + user_song_table.columns + ')'
        num_cols = len(user_song_table.columns.split(','))
        query += "VALUES (" + ('%s, ' * num_cols)[:-2] + ')'
        session.execute(query, (int(line[10]), line[1], line[4], line[9]))

#### SELECT to verify that the data have been inserted

In [18]:
query = Query(
    session=session
)

rows = query.\
    select(
        columns="userid, first_name, last_name").\
    from_table(
    table_name=user_song_table.table_name).\
    where(
    conditions="song='All Hands Against His Own'").\
    execute()

for row in rows:
    print(row.userid, row.first_name, row.last_name)

29 Jacqueline Lynch
80 Tegan Levine
95 Sara Johnson


### Drop the tables before closing out the sessions

In [19]:
songs_by_session_table.drop()
artist_song_user.drop()
user_song_table.drop()

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()